In [1]:
import sys
import os
import gc
import itertools as it
import tempfile
from collections import Counter, namedtuple
import multiprocessing as mp
import datetime as dt


import numpy as np
import pandas as pd

from joblib import Parallel, delayed, cpu_count, load, dump, Memory

%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
# corral startup
sys.path.insert(1, "/home/jbcabral/carpyncho3")
os.environ.setdefault("CORRAL_SETTINGS_MODULE", "carpyncho.settings")

from corral import core
core.setup_environment()

from corral import db
from carpyncho.models import LightCurves
# from carpyncho.steps.features_extractor import CMP

from carpyncho.lib.beamc import (
    extinction, knnfix, add_columns, MIN_BOX_SIZE, SERVER_SOURCES_LIMIT)

In [24]:
with db.session_scope() as session:
    lc = session.query(LightCurves).filter(LightCurves.tile.has(name="b396")).one()
    feats = lc.features
    

[carpyncho-sqlalchemy.engine.base.Engine-INFO@2018-10-08 21:23:44,675] BEGIN (implicit)
[carpyncho-sqlalchemy.engine.base.Engine-INFO@2018-10-08 21:23:44,678] SELECT "LightCurves".src_obs_cnt AS "LightCurves_src_obs_cnt", "LightCurves".id AS "LightCurves_id", "LightCurves".tile_id AS "LightCurves_tile_id", "Tile_1".raw_filename AS "Tile_1_raw_filename", "Tile_1".npy_filename AS "Tile_1_npy_filename", "Tile_1".id AS "Tile_1_id", "Tile_1".name AS "Tile_1_name", "Tile_1".size AS "Tile_1_size", "Tile_1".status AS "Tile_1_status", "Tile_1".ogle3_tagged_number AS "Tile_1_ogle3_tagged_number", "Tile_1".ready AS "Tile_1_ready" 
FROM "LightCurves" LEFT OUTER JOIN "Tile" AS "Tile_1" ON "Tile_1".id = "LightCurves".tile_id 
WHERE EXISTS (SELECT 1 
FROM "Tile" 
WHERE "Tile".id = "LightCurves".tile_id AND "Tile".name = %(name_1)s)
[carpyncho-sqlalchemy.engine.base.Engine-INFO@2018-10-08 21:23:44,680] {'name_1': 'b396'}
[carpyncho-sqlalchemy.engine.base.Engine-INFO@2018-10-08 21:24:17,176] COMMIT


In [25]:
feats = pd.DataFrame(feats)

In [28]:
nbgs = pd.read_pickle("data/all_nbgs.pkl")
no_nbgs = pd.read_pickle("data/all_no_nbgs.pkl")

In [29]:
src_ids = np.concatenate((
    np.unique(nbgs[["src_1", "src_2"]].values),
    np.unique(no_nbgs[["src_1", "src_2"]].values)))
src_ids = np.unique(src_ids)

In [30]:
feats = feats[feats.id.isin(src_ids)]

In [34]:
coso = feats.iloc[0]

In [55]:
rows = []
for src_1, src_2 in nbgs[["src_1", "src_2"]].values:
    row = {"src1_{}".format(k): v for k, v in feats[feats.id == src_1].iloc[0].to_dict().items()}
    row.update({"src2_{}".format(k): v for k, v in feats[feats.id == src_2].iloc[0].to_dict().items()})
    row["nbg"] = 1
    rows.append(row)
    
for src_1, src_2 in no_nbgs[["src_1", "src_2"]].values:
    row = {"src1_{}".format(k): v for k, v in feats[feats.id == src_1].iloc[0].to_dict().items()}
    row.update({"src2_{}".format(k): v for k, v in feats[feats.id == src_2].iloc[0].to_dict().items()})
    row["nbg"] = 0
    rows.append(row)   
    
    

# feats[feats.id == 33960000003398].to_dict()

In [57]:
df = pd.DataFrame(rows)

In [67]:
columns = [c for c in list(df.columns) if c not in ['src2_ogle3_type', 'src1_ogle3_type', "nbg", 'src2_id', 'src1_id']]
columns = ["src1_id", "src2_id", "nbg"] + columns

In [69]:
df = df[columns]

In [70]:
df.to_pickle("data/sample.pkl")